In [ ]:
import torch.nn as nn
import torch
import os

import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.utils import save_image
import tqdm
import torchvision
import torch.optim as optim

In [ ]:
os.makedirs("./drive/MyDrive/preds/", exist_ok=True)
os.makedirs("./drive/MyDrive/preds/A/", exist_ok=True)
os.makedirs("./drive/MyDrive/preds/B/", exist_ok=True)

In [ ]:

!rm -rf /content/drive/*

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

trA = './drive/MyDrive/selfie2anime/trainA'
trB = './drive/MyDrive/selfie2anime/trainB'
teA = './drive/MyDrive/selfie2anime/testA'
teB = './drive/MyDrive/selfie2anime/testB'



MessageError: Error: credential propagation was unsuccessful

In [ ]:
preprocess = T.Compose(
    [
        T.RandomHorizontalFlip(),
        T.Resize((128,128)),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

trainA = torchvision.datasets.ImageFolder(root=trA, transform=preprocess)
trainB = torchvision.datasets.ImageFolder(root=trB, transform=preprocess)
testA = torchvision.datasets.ImageFolder(root=teA, transform=preprocess)
testB = torchvision.datasets.ImageFolder(root=teB, transform=preprocess)

dlTrainA = torch.utils.data.DataLoader(trainA, batch_size=16,shuffle=True, num_workers=10)
dlTrainB = torch.utils.data.DataLoader(trainB, batch_size=16,shuffle=True, num_workers=10)
dlTestA = torch.utils.data.DataLoader(testA, batch_size=16, shuffle=False, num_workers=10)
dlTestB = torch.utils.data.DataLoader(testB, batch_size=16, shuffle=False, num_workers=10)

NameError: name 'trA' is not defined

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=2, padding=2),
        nn.LeakyReLU(0.2, inplace=True),
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2),
        nn.InstanceNorm2d(128),
        nn.LeakyReLU(0.2, inplace=True),
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=2, padding=2),
        nn.InstanceNorm2d(256),
        nn.LeakyReLU(0.2, inplace=True),
    )

    self.fc1 = nn.Linear(256*32*32, 1)

  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = x.reshape(-1, 256 * 32 * 32)
    x = self.fc1(x)
    out = torch.sigmoid(x)
    return out

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.conv1 = nn.Sequential(
      nn.ReflectionPad2d(2),
      nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=2, padding=0),
      nn.InstanceNorm2d(64),
      nn.ReLU(inplace=True)
    )
    self.down1 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
      nn.InstanceNorm2d(128),
      nn.ReLU(inplace=True),
      nn.Dropout(p=0.5),
      nn.ReflectionPad2d(1),
      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=0),
      nn.InstanceNorm2d(128),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.down2 = nn.Sequential(
      nn.ReflectionPad2d(1),
      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=0),
      nn.InstanceNorm2d(128),
      nn.ReLU(inplace=True),
      nn.Dropout(p=0.5),
      nn.ReflectionPad2d(1),
      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=0),
      nn.InstanceNorm2d(128),
    )

    self.up1 = nn.Sequential(
      nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2),
      nn.InstanceNorm2d(64),
      nn.ReLU(inplace=True),
    )

    self.final_conv = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=2, stride=2)

  def forward(self, x):
    x = self.conv1(x)
    x = self.down1(x)
    x = self.down2(x)
    x = self.down2(x)
    x = self.down2(x)
    x = self.down2(x)
    x = self.up1(x)
    out = self.final_conv(x)
    return out

In [ ]:
import torchvision.transforms.functional as F

def train(loader_train_a, loader_train_b, loader_test_a, loader_test_b, gen_a, gen_b, dis_a, dis_b, gen_optimizer, dis_optimizer, epochs=100):
  criterion = torch.nn.MSELoss()
  criterion_L1 = torch.nn.L1Loss()
  gen_a = gen_a.cuda()
  gen_b = gen_b.cuda()
  dis_a = dis_a.cuda()
  dis_b = dis_b.cuda()

  for e in range(epochs):
    print("epoch" + str(e))

    total_loss_a = 0
    total_loss_b = 0
    total_gen_loss_a = 0
    total_gen_loss_b = 0
    for im_a, im_b in tqdm.tqdm(zip(loader_train_a, loader_train_b), total=len(loader_train_a)):
      real_a = im_a[0].cuda()
      real_b = im_b[0].cuda()

      # train generator b cycle
      gen_optimizer.zero_grad()
      fake_a = gen_b(real_b).detach()
      pred_fake_a = dis_a(fake_a)
      loss_gen_b = criterion(pred_fake_a, torch.ones_like(pred_fake_a))
      recovered_b = gen_a(fake_a)
      loss_cycle_1 = criterion_L1(recovered_b, real_b) * 10
      b_id = gen_b(real_a)
      loss_b_id = criterion_L1(b_id, real_a) * 10
      final_loss_gen_b = loss_gen_b + loss_cycle_1 + loss_b_id
      final_loss_gen_b.backward(retain_graph=True)
      gen_optimizer.step()

      # train generator a cycle
      gen_optimizer.zero_grad()
      fake_b = gen_a(real_a).detach()
      pred_fake_b = dis_b(fake_b)
      loss_gen_a = criterion(pred_fake_b, torch.ones_like(pred_fake_b))
      recovered_a = gen_b(fake_b)
      loss_cycle_2 = criterion_L1(recovered_a, real_a) * 10
      a_id = gen_a(real_b)
      loss_a_id = criterion_L1(a_id, real_b) * 10
      final_loss_gen_a = loss_gen_a + loss_cycle_2 + loss_a_id
      final_loss_gen_a.backward(retain_graph=True)
      gen_optimizer.step()

      # train discriminator a
      # real
      dis_optimizer.zero_grad()
      pred_real_b = dis_a(real_b)
      loss_dis_a_real = criterion(pred_real_b, torch.ones_like(pred_real_b))

      # fake
      fake_b = gen_a(real_a)
      pred_fake_b = dis_a(fake_b)
      loss_dis_a_fake = criterion(pred_fake_b, torch.ones_like(pred_fake_b))

      loss_dis_a = (loss_dis_a_real + loss_dis_a_fake) / 2
      loss_dis_a.backward()
      dis_optimizer.step()

      # train discriminator b
      # real
      dis_optimizer.zero_grad()
      pred_real_a = dis_b(real_a)
      loss_dis_b_real = criterion(pred_real_a, torch.ones_like(pred_real_a))

      #fake
      fake_a = gen_b(real_b)
      pred_fake_a = dis_b(fake_a)
      loss_dis_b_fake = criterion(pred_fake_a, torch.ones_like(pred_fake_a))

      loss_dis_b = (loss_dis_b_real + loss_dis_b_fake) / 2
      loss_dis_b.backward()
      dis_optimizer.step()


      total_loss_a += loss_dis_a.item()
      total_loss_b += loss_dis_b.item()
      total_gen_loss_a += final_loss_gen_a.item()
      total_gen_loss_b += final_loss_gen_b.item()



    for test_a, test_b in tqdm.tqdm(zip(loader_test_a, loader_train_b), total=len(loader_test_a)):
      test_a = test_a[0].cuda()
      test_b = test_b[0].cuda()
      fake_test_a = gen_b(test_b)
      fake_test_b = gen_a(test_a)


      save_image(fake_test_a * 0.5 + 0.5, f"./drive/MyDrive/preds/A/anime_{e}_{d}.png")
      save_image(fake_test_b * 0.5 + 0.5, f"./drive/MyDrive/preds/B/human_{e}_{d}.png")

    print(f"Epoch [{e+1}/{epochs}], Loss D_A: {total_loss_a / 3400}, Loss D_B: {total_loss_b / 3400}, Loss G_A: {total_gen_loss_a / 3400}, Loss G_B: {total_gen_loss_b / 3400}")




In [ ]:
gen_A = Generator()
gen_B = Generator()
dis_A = Discriminator()
dis_B = Discriminator()
gen_lr = 0.0002
dis_lr = 0.0001

gen_opt = optim.Adam(list(gen_A.parameters()) + list(gen_B.parameters()), lr=gen_lr, betas=(0.5, 0.999))
dis_opt = optim.Adam(list(dis_A.parameters()) + list(dis_B.parameters()), lr=dis_lr, betas=(0.5, 0.999))


train(dlTrainA, dlTrainB, dlTestA, dlTestB, gen_A, gen_B, dis_A, dis_B, gen_opt, dis_opt, epochs=100)


epoch0


100%|██████████| 7/7 [00:08<00:00,  1.24s/it]


Epoch [1/100], Loss D_A: 4.482887778609743e-05, Loss D_B: 9.037817992099651e-05, Loss G_A: 0.4005918421464808, Loss G_B: 0.4029512235697578
epoch1


100%|██████████| 7/7 [00:03<00:00,  2.16it/s]


Epoch [2/100], Loss D_A: 2.0882642020831767e-15, Loss D_B: 5.748081749141634e-15, Loss G_A: 0.32073316546047437, Loss G_B: 0.3269041445676018
epoch2


...

100%|██████████| 7/7 [00:03<00:00,  2.08it/s]


Epoch [29/100], Loss D_A: 2.644159401236667e-15, Loss D_B: 5.137328471124313e-15, Loss G_A: 0.2028634837094475, Loss G_B: 0.2120032221429488
epoch29


100%|██████████| 7/7 [00:03<00:00,  1.81it/s]


Epoch [30/100], Loss D_A: 2.738724280040033e-15, Loss D_B: 5.335340012928046e-15, Loss G_A: 0.2023890568228329, Loss G_B: 0.21172735052950242
epoch30


 30%|███       | 66/220 [00:30<01:09,  2.23it/s]